In [14]:
# Here i am using dataset from kaggle

!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip -O /tmp/cats_and_dogs_filtered.zip

--2020-11-26 15:12:01--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 74.125.200.128, 2404:6800:4003:c04::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  10.5MB/s    in 6.2s    

2020-11-26 15:12:08 (10.5 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [15]:
# Extracting Dataset
import zipfile
import os

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [16]:
# Preparing Dataset
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, "cats")
train_dogs_dir = os.path.join(train_dir, "dogs")

testing_cats_dir = os.path.join(validation_dir, "cats")
testing_dogs_dir = os.path.join(validation_dir, "dogs")

In [17]:
print("total training cats images:", len(os.listdir(train_cats_dir)))
print("total training dogs images:", len(os.listdir(train_dogs_dir)))
print("total testing cats images:", len(os.listdir(testing_cats_dir)))
print("total testing dogs images:", len(os.listdir(testing_dogs_dir)))

total training cats images: 1000
total training dogs images: 1000
total testing cats images: 500
total testing dogs images: 500


In [18]:
from tensorflow.keras import layers
from tensorflow.keras import Model



img_input = layers.Input(shape=(150,150,3))

x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Flatten()(x)

x = layers.Dense(512, activation='relu')(x)

output = layers.Dense(1, activation='sigmoid')(x)


model = Model(img_input, output)

In [19]:
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 64)       

In [20]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss="binary_crossentropy", optimizer=RMSprop(lr=0.001),metrics=['acc'])

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(1./255)
val_datagen = ImageDataGenerator(1./255)

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(150,150),
                                                    batch_size=20,
                                                    class_mode="binary")

validation_generator = val_datagen.flow_from_directory(validation_dir, 
                                                    target_size=(150,150),
                                                    batch_size=20,
                                                    class_mode="binary")

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [22]:
history = model.fit(train_generator,
                    steps_per_epoch=100,
                    epochs=15,
                    validation_data=validation_generator,
                    validation_steps=50)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/15
100/100 [==============================] - 9s 86ms/step - loss: 51.3825 - acc: 0.5355 - val_loss: 0.7718 - val_acc: 0.5500
Epoch 2/15
100/100 [==============================] - 9s 85ms/step - loss: 3.4231 - acc: 0.6405 - val_loss: 0.7563 - val_acc: 0.6050
Epoch 3/15
100/100 [==============================] - 8s 84ms/step - loss: 1.7593 - acc: 0.6430 - val_loss: 0.7924 - val_acc: 0.5580
Epoch 4/15
100/100 [==============================] - 8s 84ms/step - loss: 1.0399 - acc: 0.6495 - val_loss: 1.3095 - val_acc: 0.5020
Epoch 5/15
100/100 [==============================] - 8s 84ms/step - loss: 0.7785 - acc: 0.7260 - val_loss: 7.5421 - val_acc: 0.5160
Epoch 6/15
100/100 [==============================] - 8s 85ms/step - loss: 0.8350 - acc: 0.7815 - val_loss: 0.8919 - val_acc: 0.6220
Epoch 7/15
100/100 [==============================] - 9s 87ms/step - loss: 0.7475 - acc: 0.7965 - val_loss: 2.5939 - val_acc: 0.5450
Epoch 8/15
100/100 [==============================] - 9s 85ms/step -

In [23]:
model.save("perfectmodel")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: perfectmodel/assets


In [24]:
!zip -r /content/perfectmodel.zip /content/perfectmodel

  adding: content/perfectmodel/ (stored 0%)
  adding: content/perfectmodel/assets/ (stored 0%)
  adding: content/perfectmodel/variables/ (stored 0%)
  adding: content/perfectmodel/variables/variables.index (deflated 65%)
  adding: content/perfectmodel/variables/variables.data-00000-of-00001 (deflated 12%)
  adding: content/perfectmodel/saved_model.pb (deflated 89%)
